<a href="https://colab.research.google.com/github/jopachecoc/NPL/blob/main/Quinta_Entrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generación de Texto con modelos GPT

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-nlp/blob/main/Sesion5/1-text-generation.ipynb)

En este notebook harémos uso de un modelo tipo GPT-2 pre-entrenado en idioma español que utilizaremos para generar texto a partir de un contexto inicial que proveerémos. Luego, harémos fine tuning a este modelo con un dataset de chistes en español y observar como cambia la generación de texto en función del dataset que utilicemos.

#### Referencias
- Dataset: https://huggingface.co/datasets/mrm8488/CHISTES_spanish_jokes
- [Improving Language Understanding by Generative Pre-Training](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)
- [Natural Language Processing with Transformers: Building Language Applications With Hugging Face](https://www.amazon.com/Natural-Language-Processing-Transformers-Applications/dp/1098103246)
- [GPT2 Spanish](https://huggingface.co/DeepESP/gpt2-spanish)
- [Fine-Tune a non-Englush GPT-2 Model with Huggingface](https://www.philschmid.de/fine-tune-a-non-english-gpt-2-model-with-huggingface)

In [36]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

In [37]:
#!test '{IN_COLAB}' = 'True' && wget  https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/requirements.txt && pip install -r requirements.txt
!test '{IN_COLAB}' = 'True' && sudo apt-get update -y
!test '{IN_COLAB}' = 'True' && sudo apt-get install python3.10 python3.10-distutils python3.10-lib2to3 -y
!test '{IN_COLAB}' = 'True' && sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.11 2
!test '{IN_COLAB}' = 'True' && sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 1
!test '{IN_COLAB}' = 'True' && pip install lightning datasets 'transformers[torch]' sentence-transformers torchinfo evaluate

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

## Generative pre-training Transformer - GPT

![](../assets/gpt.png)

Los modelos tipo GPT, introducidos por Radfor, et.al., de OpenAI, al igual que los modelos BERT, hacen uso extensivo de la arquitectura de transformers como hemos estado viendo. Las diferencias claves se podrían resumir en:

1. GPT utiliza bloques de **Transformer Decoder** encadenados, mientras que el modelo BERT utiliza bloques de *Transformer Encoder*
2. GPT se centra en la generación de texto basado en un contexto, la tarea principal es la predicción del siguiente token en la secuencia, mientras que BERT se centra en el completado de partes de una secuencia, en función de un contexto anterior y posterior a la secuencia de entrada. Entonces BERT se centra en la construicción de representación de lenguage, mientras que GPT se centra en la generación de texto en función de un contexto.

Sin embargo, ambos se basan en la misma premisa de pre-entrenar el modelo en tareas no-supervisadas o semi-supervisadas para que el modelo aprenda las representaciones semánticas del lenguage y luego al modelo se le pueda hacer fine tuning a tareas posteriores.

**Para variar un poco con el codigo existente, decidimos usar datificate/gpt2-small-spanish**:

https://huggingface.co/datificate/gpt2-small-spanish

**Intentamos con otros modelos gpt más grande pero los tiempos de entrenamiento y tuning del modelo eran absurdos. Este modelo tenia pesos parecidos al que se uso de referencia (500MB)**

In [38]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer


device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "datificate/gpt2-small-spanish"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [39]:
modules = [m for m, _ in model.named_modules()]
modules

['',
 'transformer',
 'transformer.wte',
 'transformer.wpe',
 'transformer.drop',
 'transformer.h',
 'transformer.h.0',
 'transformer.h.0.ln_1',
 'transformer.h.0.attn',
 'transformer.h.0.attn.c_attn',
 'transformer.h.0.attn.c_proj',
 'transformer.h.0.attn.attn_dropout',
 'transformer.h.0.attn.resid_dropout',
 'transformer.h.0.ln_2',
 'transformer.h.0.mlp',
 'transformer.h.0.mlp.c_fc',
 'transformer.h.0.mlp.c_proj',
 'transformer.h.0.mlp.act',
 'transformer.h.0.mlp.dropout',
 'transformer.h.1',
 'transformer.h.1.ln_1',
 'transformer.h.1.attn',
 'transformer.h.1.attn.c_attn',
 'transformer.h.1.attn.c_proj',
 'transformer.h.1.attn.attn_dropout',
 'transformer.h.1.attn.resid_dropout',
 'transformer.h.1.ln_2',
 'transformer.h.1.mlp',
 'transformer.h.1.mlp.c_fc',
 'transformer.h.1.mlp.c_proj',
 'transformer.h.1.mlp.act',
 'transformer.h.1.mlp.dropout',
 'transformer.h.2',
 'transformer.h.2.ln_1',
 'transformer.h.2.attn',
 'transformer.h.2.attn.c_attn',
 'transformer.h.2.attn.c_proj',
 'tran

Observermos un ejemplo de generación simple.

In [40]:
text = "Había una vez"
best = 10

with torch.no_grad():
    tokens = tokenizer(text, return_tensors='pt')['input_ids'].to(device)
    print("Dimensiones de la entrada:", tokens.shape)
    output = model(input_ids=tokens)
    print("Dimensiones de la salida:", output.logits.shape)
    output = output.logits[0, -1, :]
    print("Dimensiones del último token de la secuencia:", output.shape)
    probs = torch.softmax(output, dim=-1)
    print("Dimensiones de la probabilidad de los tokens:", probs.shape)
    sorted_probs = torch.argsort(probs, dim=-1, descending=True)
    print({tokenizer.decode(token): f"{prob.cpu().numpy() * 100:.2f}%" for token, prob in zip(sorted_probs[:best], probs[sorted_probs[:best]])})

Dimensiones de la entrada: torch.Size([1, 3])
Dimensiones de la salida: torch.Size([1, 3, 50257])
Dimensiones del último token de la secuencia: torch.Size([50257])
Dimensiones de la probabilidad de los tokens: torch.Size([50257])
{' más': '60.62%', ' que': '9.39%', ' en': '2.13%', ' por': '1.26%', ' el': '1.00%', ' terminada': '0.96%', ' la': '0.90%', ' un': '0.69%', ' al': '0.66%', ' una': '0.63%'}


## Implementando una función de generación

Ahora, la idea es que este modelo nos sirva para generar texto de forma recurrente e incremental. En la última capa de los modelos tipo GPT encontrarémos un tensor con forma $(b, s, v)$, donde:

- $b$: Es el tamaño del bache, o la cantidad de secuencias a procesar.
- $s$: Es la longitud de la secuencia de entrada.
- $v$: Es el tamaño del vocabulario del modelo, cuantos tokens soporta.

Pero este es el tensor de salida, por qué tiene la forma de la secuencia de entrada?, porque cada posición en la salida corresponde a la la predicción del siguiente token de esa posición en la secuencia de entrada. En otras palabras, lo que obtenemos como predicción, es una secuencia de igual tamaño a la de entrada, movida un token hacia adelante, lo que efectivamente nos predice un solo token a la vez y por ende, el token que nos insteresa, es el último.

Lo que obtenemos en este tensor es además los logits de TODO el vocabulario del modelo, con los cuales podemos calcular las probabilidades de que cada uno sea el que continue en la secuencia. Hay varias formas de decodificar el siguiente token, la más fácil de implementar sería una decodificación codiciosa (greedy) del siguiente token, que consiste simplemente en seleccionar el token con la probabilidad más alta. Este es un enfoque simple y efectivo para algunos casos, pero al mismo tiempo sufre de poca variabilidad e incluso puede caer en generación repetitiva.

Otra opción es el muestreo, ya que justamente podemos obtener probabilidades del siguiente token, lo más lógico sería muestrear con esas opciones de probabilidad, de este modo podemos obtener mayor diversidad a la hora de generar el texto, al costo eso si de que haya mayor aleatoridad ya que se le daría la oportunidad a incluso tokens con baja probabilidad, de ser seleccionados.

Otra opción podría ser un balanceo entre una decodificación greedy y una por muestreo, en función de otro hiperparámetro que podemos definir. Esta sería una técnica muy común en el contexto de Reinforcement Learning llamade e-greedy. Se hace la aclaración de que en este ejemplo no harémos nada de RL, solamente se hace mención de esta técnica para balancear entre explotación y exploración.

In [41]:
import torch.nn as nn
import numpy as np
import pandas as pd
from typing import Optional, Tuple
from transformers.tokenization_utils_base import PreTrainedTokenizerBase


def generate(
        model: nn.Module,
        tokenizer: PreTrainedTokenizerBase,
        start: str,
        max_length: int = 100,
        eps: float = 0.5,
        top_n: int = 5,
        return_iterations: bool = False,
        device: str = "cpu") -> Tuple[str, Optional[pd.DataFrame]]:

    output = [start]
    iterations = []
    with torch.no_grad():
        input_ids = tokenizer(output[-1], return_tensors='pt')['input_ids'].to(device)
        for _ in range(max_length):
            # Tomamos los logits producidos por la última capa del modelo
            # Estos corresponden al siguiente token por cada posición de la cadena
            logits = model(input_ids=input_ids).logits
            # Por lo tanto, el que nos interesa es el último, que correspondería a la
            # predicción del siguiente token después del final de la cadena original
            # A este aplicamos un softmax para obtener las probabilidades por cada
            # token del vocabulario para estar presente en la cadena.
            probs = torch.softmax(logits[0, -1, :], dim=-1)
            # Simplemente ordenamos por probabilidad de forma descendente
            sorted_tokens = torch.argsort(probs, dim=-1, descending=True)

            # Utilizamos una politica tipo e-greedy para obtener el siguiente token de la secuencia
            # Un eps>=1 quiere decir que siempre se va seleccionar el token de forma 'greedy', es decir
            # siempre se toma el token con probabilidad más alta.

            # Un eps=0 quiere decir que siempre se va a muestrear el siguiente token en función
            # de las probabilidades de cada token

            # Un 0<eps<1 va a balancear de forma binomial entre tomar el token con la
            # probabilidad más alta y muestrear el token en función de sus probabilidades.
            if np.random.random_sample(1)[0] < eps:
                # Se toma el mejor token
                next_token = sorted_tokens[0].unsqueeze(dim=0)
            else:
                # Se muetrea el token de la probabilidad de distribución
                next_token = torch.multinomial(probs, 1)

            if return_iterations:
                # Mantenemos pista de todas las iteraciones para análisis
                iteration = {'input': ''.join(output)}
                best_n = sorted_tokens[:top_n].cpu().tolist()
                choices = {f'Choice #{choice+1}': f'{tokenizer.decode(token)} ({prob:.4f})' for choice, (token, prob) in enumerate(zip(best_n, probs[best_n].cpu().tolist()))}
                iteration.update(choices)
                iterations.append(iteration)

            output.append(tokenizer.decode(next_token))
            input_ids = torch.cat([input_ids, next_token.unsqueeze(dim=0)], dim=-1)

        output_text = ''.join(output)
        if not return_iterations:
            return output_text, None
        else:
            df = pd.DataFrame(iterations)
            return output_text, df

Ahora observemos que pasa cuando generamos texto con nuestra función y algunos parámetros.

Primero, observemos que pasa cuando pasamos un `eps=1` que quiere decir que la generación va a ser de tipo greedy:

In [42]:
output_text, iterations_df = generate(model, tokenizer, text, max_length=15, eps=1.0, top_n=10, return_iterations=True, device=device)
print(output_text)
iterations_df.head(15)

Había una vez más en la historia de la música.

En el año 2000,


,input,Choice #1,Choice #2,Choice #3,Choice #4,Choice #5,Choice #6,Choice #7,Choice #8,Choice #9,Choice #10
0,Había una vez,más (0.6062),que (0.0939),en (0.0213),por (0.0126),el (0.0100),terminada (0.0096),la (0.0090),un (0.0069),al (0.0066),una (0.0063)
1,Había una vez más,en (0.1280),", (0.1211)",la (0.0962),. (0.0685),el (0.0540),a (0.0419),por (0.0246),una (0.0238),las (0.0222),su (0.0193)
2,Había una vez más en,la (0.2168),el (0.1995),su (0.0363),las (0.0325),los (0.0260),un (0.0218),una (0.0168),manos (0.0071),Europa (0.0067),""" (0.0057)"
3,Había una vez más en la,historia (0.0895),lista (0.0334),ciudad (0.0310),cima (0.0249),que (0.0202),línea (0.0197),vida (0.0174),serie (0.0130),capital (0.0119),temporada (0.0117)
4,Había una vez más en la historia,de (0.3537),. (0.1174),del (0.1103),", (0.1058)",y (0.0262),en (0.0241),cuando (0.0188),que (0.0184),como (0.0142),a (0.0130)
5,Había una vez más en la historia de,la (0.4002),los (0.0824),las (0.0448),México (0.0161),""" (0.0152)",este (0.0138),su (0.0133),España (0.0125),esta (0.0090),Estados (0.0088)
6,Había una vez más en la historia de la,música (0.0654),humanidad (0.0499),ciencia (0.0451),televisión (0.0384),literatura (0.0304),ciudad (0.0289),banda (0.0265),serie (0.0153),historia (0.0152),cultura (0.0113)
7,Había una vez más en la historia de la música,. (0.0933),de (0.0812),", (0.0735)",en (0.0518),popular (0.0481),y (0.0445),electrónica (0.0353),clásica (0.0323),pop (0.0260),latina (0.0216)
8,Había una vez más en la historia de la música.,\n (0.2624),En (0.0798),El (0.0604),La (0.0538),\n (0.0352),Los (0.0175),Se (0.0156),A (0.0154),Su (0.0149),Sin (0.0115)
9,Había una vez más en la historia de la música.\n,\n (0.9493),En (0.0085),El (0.0065),La (0.0052),A (0.0016),Los (0.0015),Se (0.0009),Su (0.0008),Las (0.0007),Durante (0.0007)


Observamos como el input progresa a la vez que las opciones de tokens que hay. Sin importar cuantas veces invoquemos a la función con los mismos parámetros, siempre vamos a obtener los mismos resultados.

Ahora, observemos que pasa si introducimos exploración al reducir el `eps=0.5`, lo cual nos dice que aproximadamente la mitad de las veces va a elegir el siguiente token muestreando y la otra mitad explotando.

In [43]:
output_text, _ = generate(model, tokenizer, text, max_length=100, eps=0.5, device=device)
print(output_text)

Había una vez más en el mundo. El primer ministro de la India, Ahmed Neuwad, dijo que aunque Brasil no había alcanzado la independencia, el gobierno de la India "existe los sueños de poblaciones que no han sido capaces de hacer nada" y que se había convertido en un "estado de irreversible guerra mundial".

Ya con la independencia, el Perú y China habían logrado superarse, aunque la Saddam Hussein había sido asesinado en 1988; la Unión Soviética había sido derrotada en la Guerra Civil


En este caso, cada vez que invoquemos a la función, vamos a obtener una respuesta diferente, a veces más coherente y otras veces menos. Vale la pena realizar varias pruebas para observar los resultados hasta encontrar un balance.

### Generando texto con las utilidades del modelo

Ahora, la clase de Huggingface implementa la función `generate` que hace la labor de generación por nosotros, incluyendo las opciones de muestreo y explotación como hemos observado. Solo que además permite otra serie de parámetros y opciones para controlar la generación de texto. Para más información se recomienda estudiar:

- [Natural Language Processing with Transformers: Building Language Applications With Hugging Face](https://www.amazon.com/Natural-Language-Processing-Transformers-Applications/dp/1098103246), Capitulo 5
- https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/text_generation#transformers.GenerationConfig
- https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/text_generation#transformers.GenerationMixin.generate

In [44]:
output = model.generate(tokens, pad_token_id=tokenizer.eos_token_id, max_length=100, do_sample=True, temperature=0.5, top_k=0)
print(tokenizer.decode(output[0]))

Había una vez más al norte, en el extremo sur de la península ibérica, en la costa atlántica, y en la parte de la península ibérica.

El clima es mediterráneo continentalizado, con veranos cálidos y secos, con inviernos fríos y húmedos, veranos cálidos y secos, con inviernos templados y lluviosos. La temperatura media anual es de 18 °C, la media anual es de 17 °C y la media anual es de 12 °C.

La precipitación media anual es


## Fine tuning

Ahora, intentemos hacer fine tuning a nuestro modelo. Intentemos entrenarlo en un corpus de productos de amazon en idioma español y ver como la narrativa de su output cambia.

##### Nota
A diferencia del ejemplo de referencia, este dataset es muy grande (13GB+)
#####
Tomamos solo un porcentaje del dataset para agilizar los tiempos de entrenamiento.

In [56]:
from datasets import load_dataset, DatasetDict

train_small = load_dataset("milistu/amazon-esci-data", "products", split="train[:1%]")
train_small.shuffle
dataset = DatasetDict({
    "train": train_small
})
dataset

DatasetDict({
    train: Dataset({
        features: ['product_id', 'product_title', 'product_description', 'product_bullet_point', 'product_brand', 'product_color', 'product_locale', 'split', '__index_level_0__'],
        num_rows: 13718
    })
})

In [57]:
dataset['train'][0]

{'product_id': 'B079VKKJN7',
 'product_title': '11 Degrees de los Hombres Playera con Logo, Negro, L',
 'product_description': 'Esta playera con el logo de la marca Carrier de 11 Degrees viene en negro, con el logo de la marca en el pecho y un pequeño texto en la parte posterior. La camiseta tiene cuello redondo y manga corta.',
 'product_bullet_point': '11 Degrees Negro Playera con logo\nA estrenar y genuina. Somos un vendedor autorizado de 11 Degrees.\nVer descripción del producto para obtener más información.',
 'product_brand': '11 Degrees',
 'product_color': 'Negro',
 'product_locale': 'es',
 'split': 'train',
 '__index_level_0__': 0}

In [58]:
dataset.set_format('pandas')
df = dataset['train'].to_pandas()
df.head(10)

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale,split,__index_level_0__
0,B079VKKJN7,"11 Degrees de los Hombres Playera con Logo, Ne...",Esta playera con el logo de la marca Carrier d...,11 Degrees Negro Playera con logo\nA estrenar ...,11 Degrees,Negro,es,train,0
1,B079Y9VRKS,Camiseta Eleven Degrees Core TS White (M),None,None,11 Degrees,Blanco,es,train,1
2,B07DP4LM9H,11 Degrees de los Hombres Core Pull Over Hoodi...,La sudadera con capucha Core Pull Over de 11 G...,11 Degrees Azul Core Pull Over Hoodie\nA estre...,11 Degrees,Azul,es,train,2
3,B07G37B9HP,11 Degrees Poli Panel Track Pant XL Black,None,None,11 Degrees,None,es,train,3
4,B07LCTGDHY,11 Degrees Gorra Trucker Negro OSFA (Talla úni...,None,None,11 Degrees,Negro (,es,train,4
5,B07MSD1JH3,"11 Degrees de los Hombres Optum Poly Joggers, ...",Los Optum Poly Joggers de 11 grados vienen con...,11 Degrees Negro Optum Poly Joggers\nA estrena...,11 Degrees,Negro,es,train,5
6,B07QKLGMHM,11 Degrees Core Zip Poly Top S Black,El Chándal ha sido diseñado con mangas largas ...,None,11 Degrees,Negro,es,train,6
7,B07S1VM815,11 Degrees Camiseta De Núcleo M Hot Red,None,None,11 Degrees,None,es,train,7
8,B07T1HCDXG,11 Degrees Trucker Cap - Black & White,None,None,11 Degrees,Black & White,es,train,8
9,B07VCV1LSQ,11 Degrees Chaqueta Espacial S Black,La chaqueta Space Puffer de 11 Degrees viene c...,11 Degrees Negro Chaqueta acolchada\nA estrena...,11 Degrees,Negro,es,train,9


In [59]:
df = df.dropna(subset=['product_description'])
df = df.reset_index(drop=True)
display(df.head())

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale,split,__index_level_0__
0,B079VKKJN7,"11 Degrees de los Hombres Playera con Logo, Ne...",Esta playera con el logo de la marca Carrier d...,11 Degrees Negro Playera con logo\nA estrenar ...,11 Degrees,Negro,es,train,0
1,B07DP4LM9H,11 Degrees de los Hombres Core Pull Over Hoodi...,La sudadera con capucha Core Pull Over de 11 G...,11 Degrees Azul Core Pull Over Hoodie\nA estre...,11 Degrees,Azul,es,train,2
2,B07MSD1JH3,"11 Degrees de los Hombres Optum Poly Joggers, ...",Los Optum Poly Joggers de 11 grados vienen con...,11 Degrees Negro Optum Poly Joggers\nA estrena...,11 Degrees,Negro,es,train,5
3,B07QKLGMHM,11 Degrees Core Zip Poly Top S Black,El Chándal ha sido diseñado con mangas largas ...,None,11 Degrees,Negro,es,train,6
4,B07VCV1LSQ,11 Degrees Chaqueta Espacial S Black,La chaqueta Space Puffer de 11 Degrees viene c...,11 Degrees Negro Chaqueta acolchada\nA estrena...,11 Degrees,Negro,es,train,9


In [60]:
df['Palabras por descripcion'] = df['product_description'].str.split().apply(len)
df['Palabras por descripcion'].median()

123.0

Aquí podemos observar que la mediana de longitud en terminos de palabras es de 123. Esto es esperado, pues las descripciones son de corta a mediana longitud. Por otra parte, es bastante claro que el corpus original del modelo pre-entrenado contenía texto muy diferente a este, por lo que la calidad de los resultados, sin hacer mayores modificaciones puede que no sea buena.

Sin embargo, a manera demostrativa, continuarémos con el ejercicio, prepararémos el conjunto de datos para entrenamiento.

In [61]:
def preprocess_function(max_len):
    def _preprocess_function(examples):
        return tokenizer([str(text) for text in examples['product_description']], max_length=max_len, truncation=True, padding='max_length')
    return _preprocess_function


Los modelos GPT no solo esperan `input_ids`, investigamos y resulta que incluir `attention_mask` y `labels` podia mejorar el performance.

In [62]:
dataset.reset_format()
tokenized_dataset = dataset['train'].map(preprocess_function(max_len=512), batched=True)
tokenized_dataset = tokenized_dataset.remove_columns([col for col in tokenized_dataset.column_names if col not in ['input_ids', 'attention_mask', 'labels']])
tokenized_dataset = tokenized_dataset.train_test_split(train_size=0.9)
tokenized_dataset.set_format('torch')
tokenized_dataset


Map:   0%|          | 0/13718 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 12346
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1372
    })
})

Finalmente procedemos a definir el entrenamiento. Observaremos que es muy similar a como entrenamos a BERT.

In [63]:
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments


batch_size = 8 if IN_COLAB else 2
logging_steps = len(tokenized_dataset['train']) // batch_size
# Definimos los parámetros globales de entrenamiento
training_args = TrainingArguments(
    output_dir='./hf-gpt',
    overwrite_output_dir=True,
    num_train_epochs=10,
    learning_rate=2e-5,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    disable_tqdm=False,
    logging_steps=logging_steps,
    report_to='none'
)

# Y definimos el entrenador, especificando el modelo, datasets y el tokenizador
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer
)

In [64]:
%%time
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 0, 'bos_token_id': 0, 'pad_token_id': 0}.


Epoch,Training Loss,Validation Loss
1,3.312300,3.022130
2,2.962300,2.882756
3,2.802600,2.813442
4,2.682200,2.768518
5,2.608000,2.739257
6,2.550000,2.711107
7,2.518500,2.695002
8,2.479800,2.687277
9,2.464300,2.679932
10,2.439400,2.676479


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


CPU times: user 4h 51min 37s, sys: 44.9 s, total: 4h 52min 21s
Wall time: 5h 8min 49s


TrainOutput(global_step=15440, training_loss=2.6816506641516415, metrics={'train_runtime': 18527.5468, 'train_samples_per_second': 6.664, 'train_steps_per_second': 0.833, 'total_flos': 3.225911427072e+16, 'train_loss': 2.6816506641516415, 'epoch': 10.0})

Ahora observemos los resultados.

In [65]:
output = model.generate(tokens, pad_token_id=tokenizer.eos_token_id, max_length=100, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Había una vez que este artículo se vende en el mercado, el vendedor debe pagar un precio razonable por la compra. Gracias por entender. <br /><br /> <b>Durable Función: </b> <br /> <p><b>✔Ideal para cualquier ocasión:</b> <br /> <p><b>Temporada: invierno, verano <br /> <br /> <p>Temporada: Otoño


In [66]:
output_text, _ = generate(model, tokenizer, text, max_length=100, eps=0.2, device=device)
print(output_text)

Había una vez que la pelota de fútbol se ha verificado, el balón parece ser un juguete, como si fuera una bola de madera. Pero el cable aéreo de suspensión de la pelota es realmente súper fuerte y sostenible. Un cable Bobby tiene tanto antena fija como antena doble, distingue claramente entre pies y manos. Un cable Bobby de madera, resistente a los golpes, de alta calidad a prueba de golpes, con un diseño muy sencillo y duradero, seguimos las instrucciones de disminución después de cada uso. Son muy resistentes


Se ve claramente el contexto de descripción de productos en la narrativa. Esto es un indicio de la influencia que tiene el conjunto de datos de entrenamiento en el modelo final, esto es algo a tener muy en cuenta a la hora de utilizar y hacer fine tuning a modelos de lenguaje.

## Conclusiones de referencia
- Los modelos BERT y GPT son muy similares, aunque ambos tienen diferencias claras en cuanto a su estructura y manera de entrenamiento.
- Sin embargo, ambos pueden utilizarse para el mismo tipo de tareas posteriores, lo cual sustenta la importancia del pre-entrenamiento y la construcción de embeddings de buena calidad.
- En los modelos generativos, tiende a ver un dilema de tipo exploración-explitación, al explotar los resultados, podemos ser más precisos, per al mismo tiempo más monótonos, mientras que explorando podemos ser más creativos y diversos, pero al mismo tiempo terminar con texto incoherente, difuso o alucinante. Es necesario evaluar la tarea a la mano para escoger el ajuste adecuado entre estas dos técnicas de decodificación.
- Los modelos generativos de texto no son más que una gran probabilidad de distribución y esta a su vez es completamente dependiente de los datos con los que fue entrenada. Es aquí donde se hace sumamente importante obtener y curar los conjuntos de datos con los que se entrena, de lo contrario se puede terminar con un modelo de mala calidad para la tarea en especifico.
- Diferentes estrategias de decodificación entregan resultados diferentes, vale la pena hacer una exploración de los resultados y ajustar los hiperparámetros para obtener los resultados deseados según el objetivo.
## Conclusiones propias
- El tamaño del dataset es lo mas relevante en cuanto a tiempo de entrenamiento. Solo usando el 1% del dataset nos tardo 5 horas en el fine tuning
- Fueron pocos los cambios que se hicieron con el codigo de referencia para lograr lo mismo con otro contexto. Esto es muy interesante sin duda, como con poco esfuerzo se puede reutilizar este codigo.
- Hay muchas variables que nos hubiera gustado probar en el fine tuning, pero debido a los largos tiempos de ejecución (5 horas por run), nos vimos en la obligación de quedarnos con los hiperparametros productos de solo unas cuantas ejecuciones.